### **imports**

In [1]:
import pandas as pd
import numpy as np
from astropy.io import ascii
from astropy.table import QTable, Table, Column
from astropy import units as u
import string
import os
import upsilon

-------------------
| pyFFTW detected |
-------------------


### **making a table of all of the stars in the txt file**

In [2]:
column1 = []
column2 = []
column3 = []
column4 = []
column5 = []

with open('rrlyr_vsx_clean2.txt', 'r') as file:
    for line in file:
        components = line.split(' ', 4)
    
        try:
            magnitude = float(components[3])  # Try to convert the 4th component to float
        except ValueError:
            continue

        if magnitude < 15.5:
            column1.append(components[0])
            column2.append(float(components[1]))
            column3.append(float(components[2]))
            column4.append(magnitude)
            column5.append(components[4].strip())

final_table = QTable([column1, column2, column3, column4, column5], names = ("Star Type", "RA", "Dec", "Magnitude", "Other Info"))

In [3]:
final_table

Star Type,RA,Dec,Magnitude,Other Info
str24,float64,float64,float64,str29
RRAB,0.01521,35.36286,12.43,GM And
RRAB,0.01669,18.40698,15.171,CSS_J000004.0+182425
RRAB,0.03171,34.67397,14.95,SERIV 27
RRAB,0.06821,-72.77875,15.1,BP Tuc
RRAB,0.099,36.3287,14.46,NSVS 6313844
RRAB,0.1338,22.99388,14.671,CSS_J000032.1+225937
RRAB,0.13423,-54.29512,15.26,ASASSN-V J000032.21-541742.4
RRAB,0.14829,26.66375,12.96,GV Peg
RRC,0.15377,-60.98439,15.18,ASASSN-V J000036.92-605903.9


### **importing the asassn client**

In [4]:
from pyasassn.client import SkyPatrolClient	
client = SkyPatrolClient()

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.



### **downloading a csv file with the data from the star at a specified index**

In [5]:
def classification_model(path, ra, dec, magnitude1, vsx_id, asassn_id):
  # Load a classification model.
  # (takes time, do only once if processing multiple lcs)
  rf_model = upsilon.load_rf_model()


  ############# Read input lc data #############
  # Read the input lightcurve file
  f = open(path, 'r')
  data = np.loadtxt(f, dtype={'names': ('JD', 'mag', 'err'),'formats': ('float64', 'float64', 'float64')})
  f.close()

  # Split it into three arrays
  date=data['JD']
  mag=data['mag']
  err=data['err']

  # Sigma-clip the input lightcurve
  date, mag, err = upsilon.utils.sigma_clipping(date, mag, err,  threshold=3, iteration=1)

  ############# Extract lightcurve features #############
  e_features = upsilon.ExtractFeatures(date, mag, err)
  e_features.run()
  features = e_features.get_features()

  ############# Classify #############
  # Classify the light curve
  label, probability, flag = upsilon.predict(rf_model, features)

  # Print results

  output_file = open(path.replace('.txt', '_classification.txt'), 'w')
  output_file.write("\n\n######### Classification results #########\n")
  output_file.write("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)\n")
  output_file.write("class = " + str(label) + "\n")
  output_file.write("class_probability = " + str(probability) + "\n")
  output_file.write("position = " + str(ra) + " " + str(dec) + "\n")
  output_file.write("magnitude = " + str(magnitude1) + "\n")
  output_file.write("vsx_id = " + str(vsx_id) + "\n")
  output_file.write("asassn_id = " + str(asassn_id) + "\n")
  output_file.close()

  print("\n\n######### Classification results #########\n")
  print("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)")
  print("class = " + str(label) )
  print("class_probability = " + str(probability) )
  print("\n\n######### Lightcurve features #########\n")

  for feature in features:
    print( str(feature) + " = " + str(features[feature]) )
  #  print feature," = ",features[feature]

In [8]:
folder_path = "/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/"

In [18]:
num_iterations = 1
empty_list = []
for i in range(22, 32):
    if i == 8:
        continue
    search = client.cone_search(ra_deg = final_table["RA"][i], dec_deg = final_table["Dec"][i], radius=0.0003, catalog='aavsovsx', download=True)

    savecsv = search.save(save_dir = folder_path, file_format="csv")[1]
    empty_list.append(savecsv)

    new_file = pd.read_csv(savecsv, skiprows=1)

    g_filter = new_file[new_file['phot_filter'] == 'g']
    web_format = g_filter[["jd", "mag", "mag_err"]]
    web_format.to_csv(savecsv.replace(".csv", ".txt"), sep='\t', index=False, header = False)

    classification_model(savecsv.replace(".csv", ".txt"), final_table["RA"][i], final_table["Dec"][i], final_table["Magnitude"][i], final_table["Other Info"][i], new_file["asas_sn_id"][0])

Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.3578559194440024
cusum = 0.19437883553230248
eta = 1.8144527324160695
hl_amp_ratio = 0.37841778626459055
kurtosis = -0.4932732192268854
n_points = 474
period = 0.5226797498201469
period_SNR = 114.34159285835382
period_log10FAP = -55.23578197999286
period_uncertainty = 0.0001175089424996778
phase_cusum = 0.33288814144704365
phase_eta = 0.5007709080327583
phi21 = 3.729865747967814
phi31 = 4.694749469357085
quartile31 = 0.47782938372161965
r21 = 0.4052683640914904
r31 = 0.2812904283313019
shapiro_w = 0.9388086795806885
skewness = -0.637818363838849
slope_per10 = -0.04989320472117837
slope_per90 = 0.06740197501055324
stetson_k = 0.6850163974425751
weighted_mean = 16.132214261484854
weighted_std = 0.31692470030396885
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.2816974245870616
cusum = 0.19687912149231207
eta = 2.2179247774137085
hl_amp_ratio = 0.70082754635357
kurtosis = -0.9041858225131936
n_points = 1120
period = 0.7162757703511912
period_SNR = 181.56375232651894
period_log10FAP = -159.21523410476755
period_uncertainty = 0.00022260722248024223
phase_cusum = 0.3223926761942543
phase_eta = 0.3059993014451714
phi21 = 0.9698187914918608
phi31 = 2.198537065487421
quartile31 = 0.39127033057365246
r21 = 0.4602613208719272
r31 = 0.2557660515559555
shapiro_w = 0.9614452719688416
skewness = -0.3468288041919201
slope_per10 = -0.06486742791948318
slope_per90 = 0.0675191354929587
stetson_k = 0.7804079198948897
weighted_mean = 14.619101619407722
weighted_std = 0.22600189574000287
Pulled 1 of 1


...



######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = NonVar
class_probability = 0.41


######### Lightcurve features #########

amplitude = 0.217051030580504
cusum = 0.1502357115925691
eta = 3.0862164982660127
hl_amp_ratio = 0.527427869881265
kurtosis = 11.740063501628445
n_points = 1096
period = 0.6375620164831584
period_SNR = 124.89234059951805
period_log10FAP = -61.536684571271174
period_uncertainty = 0.0001754436061534026
phase_cusum = 0.2166340604527311
phase_eta = 2.2470180341665893
phi21 = 0.9208926758225926
phi31 = -1.2993503137877478
quartile31 = 0.2962149387158135
r21 = 0.3585657079844812
r31 = 0.29098866161370224
shapiro_w = 0.8170841932296753
skewness = 1.669953779828974
slope_per10 = -0.04689401562258232
slope_per90 = 0.052539702372914714
stetson_k = 0.7376108460977834
weighted_mean = 14.016832290089692
weighted_std = 0.22137909885868862
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.26403588091132374
cusum = 0.12156223223534074
eta = 1.9575755460981463
hl_amp_ratio = 0.5193599740943874
kurtosis = -0.8510438382564116
n_points = 1714
period = 0.6263047258072663
period_SNR = 261.17285559360323
period_log10FAP = -242.57395367317758
period_uncertainty = 0.0001699605893974998
phase_cusum = 0.2770304212975489
phase_eta = 0.2965150867403576
phi21 = 0.8645315412906511
phi31 = 2.082337116116318
quartile31 = 0.35981341503529407
r21 = 0.4340427556589501
r31 = 0.2515803186956928
shapiro_w = 0.9573363661766052
skewness = -0.3849603251647678
slope_per10 = -0.03224398101699912
slope_per90 = 0.028381443216173148
stetson_k = 0.7544553879241495
weighted_mean = 15.061558998043674
weighted_std = 0.21741010256557547
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 1 (0 - classification success, 1 - suspicious period)
class = NonVar
class_probability = 0.95


######### Lightcurve features #########

amplitude = 0.007265117168731676
cusum = 0.10181671318612016
eta = 2.5312461143977134
hl_amp_ratio = 0.765042410735131
kurtosis = 0.4929494251560498
n_points = 1100
period = 0.9977502288220532
period_SNR = 22.275136478537476
period_log10FAP = -5.018873688823101
period_uncertainty = 0.000481694783412534
phase_cusum = 0.09093518431143502
phase_eta = 2.417113861387148
phi21 = 0.5736140896738697
phi31 = 2.7776222511429065
quartile31 = 0.030762445207798805
r21 = 0.10624537763517654
r31 = 0.19354241021235216
shapiro_w = 0.9909902811050415
skewness = -0.2800680145223931
slope_per10 = -0.17463526648701494
slope_per90 = 0.2638376513265947
stetson_k = 0.7816318815551636
weighted_mean = 14.147707489278027
weighted_std = 0.021820108055007583
